Run three experiments. One with no class imbalance measures, one with resampling and one with class_weights. Mease training history and compute f1-metrics

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import datetime
import time
import os
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# Some stuff to make utils-function work
import sys
sys.path.append('../utils')
from pipeline import create_dataset, split_and_create_dataset, prepare_for_training
from create_model import create_model, create_callbacks
from utils import write_to_file, get_class_weights
%load_ext autoreload
%autoreload 2

# Jupyter-specific
%matplotlib inline

project_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [2]:
data_dir = pathlib.Path('/home/henriklg/master-thesis/data/hyper-kvasir/labeled_ttv/')
unlab_dir = pathlib.Path('/home/henriklg/master-thesis/data/hyper-kvasir/unlabeled_ttv/')

dir_name = "none"
experiment = "_resample"
log_dir = "./logs/{}{}/{}".format(project_time, experiment, dir_name)

conf = {
    # Dataset
    "data_dir": data_dir,
    "unlab_dir": unlab_dir,
    "ds_info": 'hypkva',
    "augment": ["crop","flip","brightness","saturation","contrast","rotate"],
    "aug_mult": 0.5,
    "resample": False,
    "class_weight": False,
    "shuffle_buffer_size": 2000,        # 0=no shuffling
    "seed": 2511,
    "neg_class": None,                 # select neg class for binary ds (normal class)
    "outcast": None,                   # list of folders to drop - currently only works for 1 item
    # Model
    "model": 'EfficientNetB0',
    "weights": None,                   # which weights to initialize the model with
    "dropout": 0.2,
    "num_epochs": 5,
    "batch_size": 16,
    "img_shape": (128, 128, 3),
    "learning_rate": 0.001,
    "optimizer": 'Adam',
    "final_activation": 'softmax',     # sigmoid for binary ds
    # Callbacks
    "tensorboard": False,
    "learning_schedule": False,
    "decay_rate": 0,                   # 128:0.25   64:1.0   32:4.0   16:16   8:64
    "checkpoint": False,
    "early_stopp": False,
    "early_stopp_patience": 7,
    # Misc
    "verbosity": 0,
    "keep_threshold": 0.0,
    "log_dir": log_dir,
    "cache_dir": "./cache"
    }

In [3]:
ds = create_dataset(conf)

history_list = []
evaluate_list = []

## No resampling or class weight

In [5]:
model = create_model(conf)
callbacks = create_callbacks(conf)

history = model.fit(
        ds["train"],
        steps_per_epoch = conf["steps"]["train"],
        epochs = conf["num_epochs"],
        validation_data = ds["test"],
        validation_steps = conf["steps"]["test"],
        validation_freq = 1,
        callbacks = callbacks,
        verbose = 1
)

# Save the metrics from training
write_to_file(history.history, conf, "history")
write_to_file(conf, conf, "conf")
history_list.append(history.history)
with open(conf["log_dir"]+"/history_list.pkl", 'wb') as f:
    pickle.dump(history_list, f)
    
model_evaluation = model.evaluate(ds["val"], verbose=2, steps=conf["steps"]["val"])
write_to_file(model_evaluation, conf, "evaluate_val")
evaluate_list.append(model_evaluation)

Train for 465 steps, validate for 99 steps
Epoch 1/5
465/465 [==============================] - 29s 63ms/step - loss: 1.9822 - sparse_categorical_accuracy: 0.3590 - val_loss: 2.1699 - val_sparse_categorical_accuracy: 0.4009
Epoch 2/5
465/465 [==============================] - 22s 48ms/step - loss: 1.3189 - sparse_categorical_accuracy: 0.5476 - val_loss: 1.1547 - val_sparse_categorical_accuracy: 0.6054
Epoch 3/5
465/465 [==============================] - 22s 48ms/step - loss: 1.0479 - sparse_categorical_accuracy: 0.6312 - val_loss: 0.8241 - val_sparse_categorical_accuracy: 0.7033
Epoch 4/5
465/465 [==============================] - 22s 48ms/step - loss: 0.9423 - sparse_categorical_accuracy: 0.6618 - val_loss: 0.7730 - val_sparse_categorical_accuracy: 0.7279
Epoch 5/5
465/465 [==============================] - 23s 49ms/step - loss: 0.8546 - sparse_categorical_accuracy: 0.6913 - val_loss: 0.7919 - val_sparse_categorical_accuracy: 0.7254
100/100 - 1s - loss: 0.7830 - sparse_categorical_acc

## Class Weights

In [6]:
log_dir = "./logs/{}{}/{}".format(project_time, experiment, "class_weight")
pathlib.Path(log_dir).mkdir(parents=True, exist_ok=True)
conf["log_dir"] = log_dir

conf["class_weight"] = True


model = create_model(conf)
callbacks = create_callbacks(conf)
class_weights = get_class_weights(ds["train"], conf)

history = model.fit(
        ds["train"],
        steps_per_epoch = conf["steps"]["train"],
        epochs = conf["num_epochs"],
        validation_data = ds["test"],
        validation_steps = conf["steps"]["test"],
        validation_freq = 1,
        callbacks = callbacks,
        class_weight = class_weights,
        verbose = 1
)

# Save the metrics from training
write_to_file(history.history, conf, "history")
write_to_file(conf, conf, "conf")
history_list.append(history.history)
with open(conf["log_dir"]+"/history_list.pkl", 'wb') as f:
    pickle.dump(history_list, f)
    
model_evaluation = model.evaluate(ds["val"], verbose=2, steps=conf["steps"]["val"])
write_to_file(model_evaluation, conf, "evaluate_val")
evaluate_list.append(model_evaluation)

Train for 465 steps, validate for 99 steps
Epoch 1/5
465/465 [==============================] - 29s 62ms/step - loss: 4.8000 - sparse_categorical_accuracy: 0.1594 - val_loss: 5.9687 - val_sparse_categorical_accuracy: 0.0246
Epoch 2/5
465/465 [==============================] - 23s 49ms/step - loss: 4.0735 - sparse_categorical_accuracy: 0.2005 - val_loss: 3.6481 - val_sparse_categorical_accuracy: 0.2601
Epoch 3/5
465/465 [==============================] - 23s 50ms/step - loss: 4.2222 - sparse_categorical_accuracy: 0.1667 - val_loss: 7.4330 - val_sparse_categorical_accuracy: 0.1622
Epoch 4/5
465/465 [==============================] - 23s 50ms/step - loss: 4.2615 - sparse_categorical_accuracy: 0.1413 - val_loss: 7.5512 - val_sparse_categorical_accuracy: 0.1679
Epoch 5/5
465/465 [==============================] - 23s 50ms/step - loss: 4.0946 - sparse_categorical_accuracy: 0.1575 - val_loss: 3.7061 - val_sparse_categorical_accuracy: 0.2513
100/100 - 2s - loss: 2.2504 - sparse_categorical_acc

## Resampling

In [7]:
log_dir = "./logs/{}{}/{}".format(project_time, experiment, "resampling")
pathlib.Path(log_dir).mkdir(parents=True, exist_ok=True)
conf["log_dir"] = log_dir

conf["resample"] = True
conf["class_weight"] = False
ds = create_dataset(conf)


model = create_model(conf)
callbacks = create_callbacks(conf)

history = model.fit(
        ds["train"],
        steps_per_epoch = conf["steps"]["train"],
        epochs = conf["num_epochs"],
        validation_data = ds["test"],
        validation_steps = conf["steps"]["test"],
        validation_freq = 1,
        callbacks = callbacks,
        verbose = 1
)

# Save the metrics from training
write_to_file(history.history, conf, "history")
write_to_file(conf, conf, "conf")
history_list.append(history.history)
with open(conf["log_dir"]+"/history_list.pkl", 'wb') as f:
    pickle.dump(history_list, f)
    
model_evaluation = model.evaluate(ds["val"], verbose=2, steps=conf["steps"]["val"])
write_to_file(model_evaluation, conf, "evaluate_val")
evaluate_list.append(model_evaluation)

Train for 465 steps, validate for 99 steps
Epoch 1/5
465/465 [==============================] - 28s 60ms/step - loss: 2.5457 - sparse_categorical_accuracy: 0.1749 - val_loss: 2.7383 - val_sparse_categorical_accuracy: 0.2172
Epoch 2/5
465/465 [==============================] - 22s 47ms/step - loss: 1.8708 - sparse_categorical_accuracy: 0.3165 - val_loss: 1.7886 - val_sparse_categorical_accuracy: 0.4508
Epoch 3/5
465/465 [==============================] - 22s 48ms/step - loss: 1.5493 - sparse_categorical_accuracy: 0.4290 - val_loss: 1.1022 - val_sparse_categorical_accuracy: 0.6193
Epoch 4/5
465/465 [==============================] - 22s 48ms/step - loss: 1.3931 - sparse_categorical_accuracy: 0.4819 - val_loss: 1.3659 - val_sparse_categorical_accuracy: 0.5259
Epoch 5/5
465/465 [==============================] - 22s 48ms/step - loss: 1.2415 - sparse_categorical_accuracy: 0.5378 - val_loss: 1.7787 - val_sparse_categorical_accuracy: 0.4545
100/100 - 1s - loss: 1.7580 - sparse_categorical_acc

## Get previous results

# Plotting

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
x = range(conf["num_epochs"])
legends = ["none", "class_weight", "resample"]

# Plot train-val accuracy and loss
plt.figure(figsize=(14, 6))

# Subplot 1
plt.subplot(1, 2, 1)
for hist in history_list:
    plt.plot(x, hist['val_sparse_categorical_accuracy'])
plt.legend(legends, loc='lower right')
plt.ylim([0, 1])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')

# Subplot 2
plt.subplot(1, 2, 2)
for hist in history_list:
    plt.plot(x, hist['val_loss'])
plt.legend(legends, loc='upper right')
plt.ylim([0.0, 3])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Validation Loss')

plt.tight_layout()
plt.savefig('figures/resample_vs_classweight.pdf', format='pdf')
plt.show()